In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,month,date_sub,current_date,max,min
from pyspark.sql import Window
import pandas as pd

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)


In [ ]:
# ## TABLA SALES
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/Completo_ventas/Ds_G_sales.parquet'
# sales = spark.read.parquet(inpath)
# #print(sales.count())
# #display(sales.limit(100))
# sales.createOrReplaceTempView("sales_vw")

## MAESTRA MEDIOS PAGO
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Maestras/Medio_pago/maestra_medios_pago.csv'
medios_pago = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')
#display(medios_pago)

## ACCOUNT PAYMENTS
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Account_Payments/Ds_B_Acc_Payments"
#prueba = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')
Bank_ID = spark.read.parquet(inpath)
print(Bank_ID.count())
#display(identifier)

5805567


In [ ]:
# Bank_ID
Bank_ID = Bank_ID.withColumn("IDENTIFIER", trim(col('IDENTIFIER'))).\
    filter((~col('FILE_NUMBER').isNull()) &
           (~col('IDENTIFIER').isNull())).\
    withColumn("REFERENCE_DATE", to_date('REFERENCE_DATE', "dd.MM.yyyy")).\
    join(medios_pago,\
        Bank_ID["PAYMENT_METHOD"] == medios_pago["Medio_Pago"],\
        "left_outer").\
    withColumnRenamed("Agrupacion_Medio_Pago", "Metodo_pago").\
    drop("Medio_pago")

# print(Bank_ID.count())
# display(Bank_ID_TC.limit(10))


Bank_ID_TC = Bank_ID.filter(col('Metodo_pago') == 'Tarjetas')
Bank_ID_TC.createOrReplaceTempView("Bank_ID_TC_vw")


Bank_ID_PSE = Bank_ID.filter(col('Metodo_pago') == 'PSE')
Bank_ID_PSE.createOrReplaceTempView("Bank_ID_PSE_vw")

In [ ]:
tmp = Bank_ID_PSE.filter(col('IDENTIFIER') == '288836401')
display(tmp)
#'979827002	661333369	661333369	PSE - BBVA Colombia'

FILE_NUMBER,IDENTIFIER,PAYMENT_METHOD,REFERENCE_DATE,PAYMENT_AMT_ITX,Metodo_pago
12843171,288836401,Internet Safetypay,2023-08-19,1205807.0,PSE


In [ ]:
# MAESTRA BANK_ID_TC
Bank_ID_TC = spark.sql("""
with temp1 as (
	select 
	FILE_NUMBER,
	IDENTIFIER,
	sum(PAYMENT_AMT_ITX) recaudo
	from Bank_ID_TC_vw
	group by FILE_NUMBER,IDENTIFIER)

select distinct 
FILE_NUMBER,
FIRST_VALUE(IDENTIFIER) OVER (PARTITION BY FILE_NUMBER ORDER BY recaudo desc) AS IDENTIFIER
from temp1
order by FILE_NUMBER
""")

# vista de dim_payments
#dim_payments.createOrReplaceTempView("dim_payments_vw")
#Bank_ID.count()


# MAESTRA BANK_ID_PSE
Bank_ID_PSE = spark.sql("""
with temp1 as (
	select 
	FILE_NUMBER,
	IDENTIFIER,
	sum(PAYMENT_AMT_ITX) recaudo
	from Bank_ID_PSE_vw
	group by FILE_NUMBER,IDENTIFIER)

select distinct 
FILE_NUMBER,
FIRST_VALUE(IDENTIFIER) OVER (PARTITION BY FILE_NUMBER ORDER BY recaudo desc) AS IDENTIFIER
from temp1
order by FILE_NUMBER
""")

# vista de dim_payments
#dim_payments.createOrReplaceTempView("dim_payments_vw")
#Bank_ID.count()

In [ ]:
# from pyspark.sql import Window

# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("FILE_NUMBER").orderBy("FILE_NUMBER")

# df_spark = Bank_ID_PSE.withColumn("es_duplicado", count("FILE_NUMBER").over(window_spec) > 1)


# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado")#.select('Evento').distinct()

# display(duplicados)

In [ ]:
# ## TABLA AUDIENCIAS
# inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Audiencias/Ds_G_audiencias.parquet"
# audiencias = spark.read.parquet(inpath)

# tmp = audiencias.filter(col('FILE_NUMBER') == '10316317').distinct()
# display(tmp)

In [ ]:
# IMPORTACION DE DATA DE CYBER SOURCE

## febrero_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/FEBRERO_2023.csv'
febrero_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')
#display(febrero_2023.limit(10))

## marzo_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/MARZO 2023.csv'
marzo_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## abril_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/ABRIL 2023.csv'
abril_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## mayo_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/MAYO 2023.csv'
mayo_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## junio_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/JUNIO 2023.csv'
junio_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## julio_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/JULIO 2023.csv'
julio_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## agosto_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/AGOSTO 2023.csv'
agosto_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## septiembre_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/SEPTIEMBRE 2023.csv'
septiembre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## octubre_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/OCTUBRE 2023.csv'
octubre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## octubre_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/OCTUBRE 2023.csv'
octubre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## noviembre_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/NOVIEMBRE 2023.csv'
noviembre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## diciembre_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/DICIEMBRE 2023.csv'
diciembre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')

## enero_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/enero 2024.csv'
enero_2024 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## febrero_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/febrero 2024.csv'
febrero_2024 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## marzo_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/marzo 2024.csv'
marzo_2024 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## abril_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/abril 2024.csv'
abril_2024 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## mayo_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/mayo 2024.csv'
mayo_2024 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## junio_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/CyberSource/junio 2024.csv'
junio_2024 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

In [ ]:
# Transformaciones
columnas = ["MerchantReferenceNumber","CardType","BinIssuer"]

## Seleccion de columnas ###
febrero_2023 = febrero_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
marzo_2023 = marzo_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
abril_2023 = abril_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
mayo_2023 = mayo_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
junio_2023 = junio_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
julio_2023 = julio_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
agosto_2023 = agosto_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
septiembre_2023 = septiembre_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
octubre_2023 = octubre_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
noviembre_2023 = noviembre_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
diciembre_2023 = diciembre_2023.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
enero_2024 = enero_2024.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
febrero_2024 = febrero_2024.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
marzo_2024 = marzo_2024.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
abril_2024 = abril_2024.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
mayo_2024 = mayo_2024.filter(col('OverallReasonCode') == 100).select(columnas).distinct()
junio_2024 = junio_2024.filter(col('OverallReasonCode') == 100).select(columnas).distinct()

In [ ]:
# UNION
cybersource = febrero_2023.union(marzo_2023).union(abril_2023).union(mayo_2023).union(junio_2023).union(julio_2023).union(agosto_2023).union(septiembre_2023).union(octubre_2023).union(noviembre_2023).union(diciembre_2023).\
    union(enero_2024).union(febrero_2024).union(marzo_2024).union(abril_2024).union(mayo_2024).union(junio_2024)

print(cybersource.count())

923321


In [ ]:
# tmp = cybersource.select('MerchantReferenceNumber').distinct()
# print(tmp.count())

In [ ]:
# tmp = cybersource.groupBy('CardType').agg(count("CardType").alias("count"))
# display(tmp)

In [ ]:
# tmp = cybersource.filter(col('CardType') == 'Diners Club').orderBy(col('BinIssuer').desc()) # 98 bancos no nulos
# display(tmp)
# print(tmp.count())

In [ ]:
# tmp_ = tmp.filter((col('CardType').isNotNull()) &
#            (col('BinIssuer').isNotNull()))

# display(tmp_)

In [ ]:
#print(sales.count())
#sales.createOrReplaceTempView("sales_vw")
# print(cybersource.count())
#tmp = cybersource_.select('MerchantReferenceNumber').distinct()
# print(tmp.count())

In [ ]:
from pyspark.sql import Window

# Agregar una columna que indique si es duplicado o no
window_spec = Window().partitionBy("MerchantReferenceNumber").orderBy("MerchantReferenceNumber")

df_spark = cybersource.withColumn("es_duplicado", count("MerchantReferenceNumber").over(window_spec) > 1)

# Filtrar solo las filas duplicadas
duplicados = df_spark.filter("es_duplicado").\
    drop('es_duplicado').distinct()

# Filtrar solo las filas que NO son duplicadas
no_duplicados = df_spark.filter(~col("es_duplicado")).\
    drop('es_duplicado').distinct()

print(duplicados.count())
print(no_duplicados.count())
#display(duplicados.limit(100))

248760
671586


In [ ]:
# tmp = duplicados.select('MerchantReferenceNumber').distinct()
# print(tmp.count())

In [ ]:
# tmp = duplicados.filter((col('CardType').isNull()) |
#            (col('BinIssuer').isNull()))
# display(tmp.limit(100))
# print(tmp.count())

In [ ]:
duplicados = duplicados.filter((col('CardType').isNotNull()) &
           (col('BinIssuer').isNotNull()))
#display(tmp.limit(100))
#print(tmp.count())

# tmp = duplicados.select('MerchantReferenceNumber').distinct()
# print(tmp.count())

In [ ]:
# tmp = duplicados.filter(col('MerchantReferenceNumber') == '1000032231')
# display(tmp)

In [ ]:
## SEGUNDA VALIDACION DE DUPLICADOS

# Agregar una columna que indique si es duplicado o no
window_spec = Window().partitionBy("MerchantReferenceNumber").orderBy("MerchantReferenceNumber")

df_spark = duplicados.withColumn("es_duplicado", count("MerchantReferenceNumber").over(window_spec) > 1)


# Filtrar solo las filas duplicadas
duplicados_2 = df_spark.filter("es_duplicado").\
    drop('es_duplicado').distinct()

print(duplicados_2.count())
# display(duplicados_2)

4491


In [ ]:
codigos = duplicados_2.select('MerchantReferenceNumber').distinct().collect()
#print(tmp.count())
# display(codigos)

# codigos_2 = duplicados_2.select('MerchantReferenceNumber').distinct().collect()

# Extraer y guardar los ids en una lista con list comprehention
codigos = [row.MerchantReferenceNumber for row in codigos]

#cybersource_ = cybersource.filter(~col('MerchantReferenceNumber').isin(codigos))
duplicados_ = duplicados.filter(~col('MerchantReferenceNumber').isin(codigos))


# UNO
cybersource_union = no_duplicados.union(duplicados_).\
    filter(col('MerchantReferenceNumber').isNull())

print(cybersource_union.count())

107


In [ ]:
# print(Bank_ID.count())
# print(cybersource.count())
tmp = cybersource_union.select('MerchantReferenceNumber').distinct()
print(tmp.count())

1


In [ ]:
# tmp = cybersource_union.filter(col('MerchantReferenceNumber').isNull())
# display(tmp)

In [ ]:
# MAESTRA BANK_ID_TC
Bank_ID_joined = Bank_ID.join(cybersource,\
    Bank_ID["IDENTIFIER"] == cybersource["MerchantReferenceNumber"],\
    "left_outer").\
    drop("MerchantReferenceNumber").\
    withColumnRenamed("FILE_NUMBER", "Expediente")
#display(Bank_ID.limit(10))

In [ ]:
# Replace null values with a placeholder
Bank_ID_filled = Bank_ID_joined.na.fill({'CardType': 'Desconocido', 'BinIssuer': 'Desconocido'})

#print(Bank_ID.count())
#display(Bank_ID.limit(10))

In [ ]:
# tmp = Bank_ID_filled.groupBy('CardType').agg(count("CardType").alias("count"))
# display(tmp)

# tmp = Bank_ID_filled.groupBy('BinIssuer').agg(count("BinIssuer").alias("count"))
# display(tmp)

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Bank_ID/TC/'

join_df_coalesced = Bank_ID_filled.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)
#join_df_coalesced.write.mode("overwrite").orc(path_parquet_snappy)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_S_Bank_ID_TC.parquet")

True

## PSE

### Importacion de csv's

In [ ]:
# #### TABLAS ACH
# ## ACH enero 2023
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE ENERO ach.csv'
# ACH_enero_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

# ## ACH febrero 2023
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION FEBRERO PSE ach.csv'
# ACH_febrero_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

# ## ACH marzo 2023
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE MAR ach.csv'
# ACH_marzo_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

# ## ACH abril 2023
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/Conciliación PSE ach.csv'
# ACH_abril_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

# ## ACH mayo 2023
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/ACH MAYO.csv'
# ACH_mayo_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

# ## ACH junio 2023
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE JUN COMPLETA Ach.csv'
# ACH_junio_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

In [ ]:
# display(ACH_junio_2023.limit(10))

In [ ]:
# # columnas a seleccionar
# columnas = ['Ticket ID',"Valor","Banco Originador",'Fecha_Hora']

# ACH_enero_2023 = ACH_enero_2023.withColumnRenamed(' Valor ', 'Valor').\
#     withColumnRenamed('Fecha-Hora creada', 'Fecha_Hora').\
#     select(columnas).distinct()

# ACH_febrero_2023 = ACH_febrero_2023.withColumnRenamed('Fecha-Hora creada', 'Fecha_Hora').\
#     select(columnas).distinct()

# ACH_marzo_2023 = ACH_marzo_2023.withColumnRenamed(' Valor ', 'Valor').\
#     withColumnRenamed('Fecha-Hora creada', 'Fecha_Hora').\
#     select(columnas).distinct()

# ACH_abril_2023 = ACH_abril_2023.withColumnRenamed(' Valor ', 'Valor').\
#     withColumnRenamed('Fecha-Hora', 'Fecha_Hora').\
#     select(columnas).distinct()

# ACH_mayo_2023 = ACH_mayo_2023.withColumnRenamed('Fecha-Hora creada', 'Fecha_Hora').\
#     select(columnas).distinct()

# ACH_junio_2023 = ACH_junio_2023.withColumnRenamed(' Valor ', 'Valor').\
#     withColumnRenamed('Fecha-Hora', 'Fecha_Hora').\
#     select(columnas).distinct()


# # UNION
# ACH_2023 = ACH_enero_2023.union(ACH_febrero_2023).union(ACH_marzo_2023).union(ACH_abril_2023).union(ACH_mayo_2023).union(ACH_junio_2023)



In [ ]:
# # columnas a seleccionar
# columnas = ['Ticket ID',"Valor","Banco Originador"]

# ACH_enero_2023 = ACH_enero_2023.withColumnRenamed(' Valor ', 'Valor').\
#     select(columnas).distinct()

# ACH_febrero_2023 = ACH_febrero_2023.select(columnas).distinct()

# ACH_marzo_2023 = ACH_marzo_2023.withColumnRenamed(' Valor ', 'Valor').\
#     select(columnas).distinct()

# ACH_abril_2023 = ACH_abril_2023.withColumnRenamed(' Valor ', 'Valor').\
#     select(columnas).distinct()

# ACH_mayo_2023 = ACH_mayo_2023.select(columnas).distinct()

# ACH_junio_2023 = ACH_junio_2023.withColumnRenamed(' Valor ', 'Valor').\
#     select(columnas).distinct()


# # UNION
# ACH_2023 = ACH_enero_2023.union(ACH_febrero_2023).union(ACH_marzo_2023).union(ACH_abril_2023).union(ACH_mayo_2023).union(ACH_junio_2023)

In [ ]:
# columnas = ['Ticket ID',"Valor"]
# ACH_2023 = ACH_2023.withColumn('llave_ach',concat_ws("|", *columnas))

In [ ]:
# print(ACH_2023.count())
# #display(Bank_ID.limit(10))

# tmp = ACH_2023.select('llave_ach').distinct()
# print(tmp.count())

In [ ]:
# ## SEGUNDA VALIDACION DE DUPLICADOS

# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("llave_ach").orderBy("llave_ach")

# df_spark = ACH_2023.withColumn("es_duplicado", count("llave_ach").over(window_spec) > 1)


# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado").\
#     drop('es_duplicado').distinct()

# print(duplicados.count())
# display(duplicados)

In [ ]:
# tmp = ACH_2023.filter(col('Ticket ID') == '108326')
# display(tmp)


In [ ]:
###################################### Ingenico ############################
## CONCILIACION PSE ENERO Ingenico.csv
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE ENERO Ingenico.csv'
PSE_ENERO_Ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## CONCILIACION FEBRERO PSE Ingenico.csv
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION FEBRERO PSE Ingenico.csv'
PSE_FEBRERO_Ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## CONCILIACION PSE MAR Ingenico.csv
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE MAR Ingenico.csv'
PSE_MARZO_Ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## Conciliación PSE Ingenico Abril.csv
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/Conciliación PSE Ingenico.csv'
PSE_ABRIL_Ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## CONCILIACION PSE 01 AL 15 ingenico.csv   (creo que es mayo)
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE 01 AL 15 ingenico.csv'
PSE_MAYO_1_ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## CONCILIACION PSE 16 AL 28 Ingenico.csv   (creo qe es mayo)
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE 16 AL 28 Ingenico.csv'
PSE_MAYO_2_ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## CONCILIACION PSE 29 AL 31 MAYO ingenico.csv
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE 29 AL 31 MAYO ingenico.csv'
PSE_MAYO_3_ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## CONCILIACION PSE JUN COMPLETA Ingenico
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/CONCILIACION PSE JUN COMPLETA Ingenico.csv'
PSE_JUNIO_ingenico = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')


### columnas a seleccionar ###
columnas = ["`COMERCIO.ORDEN.ID`","ID"]

### SELECCION DE VARIABLES ###
PSE_ENERO_Ingenico = PSE_ENERO_Ingenico.select(columnas).distinct()
PSE_FEBRERO_Ingenico = PSE_FEBRERO_Ingenico.select(columnas).distinct()
PSE_MARZO_Ingenico = PSE_MARZO_Ingenico.select(columnas).distinct()
PSE_ABRIL_Ingenico = PSE_ABRIL_Ingenico.select(columnas).distinct()
PSE_MAYO_1_ingenico = PSE_MAYO_1_ingenico.select(columnas).distinct()
PSE_MAYO_2_ingenico = PSE_MAYO_2_ingenico.select(columnas).distinct()
PSE_MAYO_3_ingenico = PSE_MAYO_3_ingenico.select(columnas).distinct()
PSE_JUNIO_ingenico = PSE_JUNIO_ingenico.select(columnas).distinct()


#### Union de todos los ingenico ####
BD_ingenico = PSE_ENERO_Ingenico.union(PSE_FEBRERO_Ingenico).union(PSE_MARZO_Ingenico).union(PSE_ABRIL_Ingenico).union(PSE_MAYO_1_ingenico).union(PSE_MAYO_2_ingenico).union(PSE_MAYO_3_ingenico).union(PSE_JUNIO_ingenico)

###################################### Fin Ingenico ############################


In [ ]:

############# CONSOLIDADO SAFETYPAY ENE - JUN 2023 ###########
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/SAFETYPAY_Ene_Jun.csv'
SAFETYPAY_ENE_JUN_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

In [ ]:
### Cruce de Ingenico con Safetepay ####
joined_df = BD_ingenico.join(SAFETYPAY_ENE_JUN_2023,
                                BD_ingenico.ID == SAFETYPAY_ENE_JUN_2023.Merchant_Reference_Number,how='left').\
            withColumn('Payment', regexp_replace('Payment', r'^PSE - ', '')).\
            select('`COMERCIO.ORDEN.ID`','Payment').\
            withColumnRenamed('COMERCIO.ORDEN.ID','COMER ID').\
            withColumnRenamed('Payment','Banco Originador').\
            distinct()
            
print(BD_ingenico.count())
print(joined_df.count())
#display(joined_df.limit(10))

371310
369693


In [ ]:
#display(PSE_julio_dic_2023.limit(100))

## VALIDACION DE DUPLICADOS

# Agregar una columna que indique si es duplicado o no
window_spec = Window().partitionBy("COMER ID").orderBy("COMER ID")

df_spark = joined_df.withColumn("es_duplicado", count("COMER ID").over(window_spec) > 1)

# Filtrar solo las filas duplicadas
duplicados = df_spark.filter("es_duplicado")#.\
    # drop('es_duplicado').distinct()

print(duplicados.count())
display(duplicados)

0


COMER ID,Banco Originador,es_duplicado


In [ ]:
#### TABLAS PSE
## PSE enero 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/PSE julio 2023.csv'
PSE_julio_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## PSE febrero 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/PSE agosto 2023.csv'
PSE_agosto_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## PSE septiembre 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/PSE septiembre 2023.csv'
PSE_septiembre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## PSE octubre 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/PSE octubre 2023.csv'
PSE_octubre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## PSE noviembre 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/PSE noviembre 2023.csv'
PSE_noviembre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')

## PSE diciembre 2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/bronze/Financiera/PSE/PSE diciembre 2023.csv'
PSE_diciembre_2023 = spark.read.csv(inpath,header=True, inferSchema=True, sep=';')


### columnas a seleccionar ###
columnas = ["COMER ID","Banco Originador"]

### SELECCION DE VARIABLES ###
PSE_julio_2023 = PSE_julio_2023.select(columnas)
PSE_agosto_2023 = PSE_agosto_2023.select(columnas)
PSE_septiembre_2023 = PSE_septiembre_2023.select(columnas)
PSE_octubre_2023 = PSE_octubre_2023.select(columnas)
PSE_noviembre_2023 = PSE_noviembre_2023.select(columnas)
PSE_diciembre_2023 = PSE_diciembre_2023.select(columnas)


#### Union de todos los ingenico ####
PSE_julio_dic_2023 = PSE_julio_2023.union(PSE_agosto_2023).union(PSE_septiembre_2023).union(PSE_octubre_2023).union(PSE_noviembre_2023).union(PSE_diciembre_2023).distinct()

codigos = ['0']
PSE_julio_dic_2023 = PSE_julio_dic_2023.filter(~col('COMER ID').isin(codigos))

#### Fin cruce de ingeneco con Safetepay, para traer banco de PSE

In [ ]:
#display(PSE_julio_dic_2023.limit(100))

## VALIDACION DE DUPLICADOS

# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("COMER ID").orderBy("COMER ID")

# df_spark = PSE_julio_dic_2023.withColumn("es_duplicado", count("COMER ID").over(window_spec) > 1)

# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado")#.\
#     # drop('es_duplicado').distinct()

# print(duplicados.count())
# display(duplicados)


In [ ]:
# MAESTRA base_PSE
base_PSE = joined_df.union(PSE_julio_dic_2023)
base_PSE.createOrReplaceTempView("base_PSE_vw")

base_PSE = spark.sql("""
select distinct 
`COMER ID` as COMER_ID,
FIRST_VALUE(`Banco Originador`) OVER (PARTITION BY `COMER ID` ORDER BY `Banco Originador` desc) AS Banco_Originador
from base_PSE_vw
order by COMER_ID
""")

# print(base_PSE.count())

# tmp = base_PSE.select('COMER_ID').distinct()
# print(tmp.count())

In [ ]:
# MAESTRA BANK_ID_PSE
Bank_ID_joined = Bank_ID_PSE.join(base_PSE,\
    Bank_ID_PSE["IDENTIFIER"] == base_PSE["COMER_ID"],\
    "left_outer").\
    drop("COMER_ID").\
    withColumnRenamed("FILE_NUMBER", "Expediente")

display(Bank_ID_joined.limit(10))

Expediente,IDENTIFIER,Banco_Originador
10000136,1129593975,Bancolombia
10000243,298068913,Nequi
10000304,794378472,Banco Colpatria
10000427,447286933,Bancolombia
10000441,1255059923,Nequi
10000477,1521220649,Nequi
10000529,1841207322,Banco Colpatria
10000539,1213110012,Bancolombia
10000578,1006713251,Banco Falabella
10000591,695633605,Bancolombia


In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Bank_ID/PSE/'

join_df_coalesced = Bank_ID_joined.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)
#join_df_coalesced.write.mode("overwrite").orc(path_parquet_snappy)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_S_Bank_ID_PSE.parquet")

True

In [ ]:
# #tmp = Conciliacion_PSE_ach.select(min('Fecha_Hora'))

# # Seleccionar el mínimo y máximo valor de la columna renombrada usando agg
# # tmp = Conciliacion_PSE_ach.agg(
# #     min('Fecha_Hora'),
# #     max('Fecha_Hora'))


# tmp = PSE_ach.agg(
#     min(col('Fecha_Hora')).alias('Min_Fecha_Hora'),
#     max(col('Fecha_Hora')).alias('Max_Fecha_Hora'))

# #print(tmp.count())
# display(tmp)


# #,max('Fecha-Hora')).distinct(

In [ ]:
# ##### Cruce de CONCILIACION_PSE_ENERO_Ingenico con Bank_ID
# CONCILIACION_PSE_ENERO_Ingenico_Bank_ID = CONCILIACION_PSE_ENERO_Ingenico.join(Bank_ID, 
#                                                  CONCILIACION_PSE_ENERO_Ingenico["`COMERCIO.ORDEN.ID`"] == Bank_ID["IDENTIFIER"], 
#                                                  how="left")

# ##### Cruce de CONCILIACION_FEBRERO_PSE_Ingenico con Bank_ID
# CONCILIACION_FEBRERO_PSE_Ingenico_Bank_ID = CONCILIACION_FEBRERO_PSE_Ingenico.join(Bank_ID, 
#                                                  CONCILIACION_FEBRERO_PSE_Ingenico["`COMERCIO.ORDEN.ID`"] == Bank_ID["IDENTIFIER"], 
#                                                  how="left")

# ##### Cruce de CONCILIACION_PSE_MAR_Ingenico con Bank_ID
# CONCILIACION_PSE_MAR_Ingenico_Bank_ID = CONCILIACION_PSE_MAR_Ingenico.join(Bank_ID, 
#                                                  CONCILIACION_PSE_MAR_Ingenico["`COMERCIO.ORDEN.ID`"] == Bank_ID["IDENTIFIER"], 
#                                                  how="left")



# display(CONCILIACION_PSE_MAR_Ingenico_Bank_ID)
